In [ ]:
# !pip install handprint

# handprint -a google creds/handprint_google_credentials.json

# fix protobuf
# pip uninstall protobuf
# pip install protobuf==3.20.*

In [1]:
import os
import shutil
import json 
from pprint import pprint

current_path = os.getcwd() 

input_path = current_path + r"/color_correction_output"
output_dir = current_path + r"/handprint_output"
section_output = current_path + r"/section_output"

In [42]:

def move_file(old_path, new_path):
    os.rename(old_path, new_path)
    # os.replace(old_path, new_path)
    shutil.move(old_path, new_path)


for name in os.listdir(input_path):
    command = f"handprint /o {output_dir} /e /s google {input_path}/{name} "
    print(command)
    !{command}
    
for name in os.listdir(input_path):
    if name.startswith('textoutput'):
        move_file(input_path+"/"+name, output_dir+"/"+name)

handprint /o d:\GitHub\PWS-inscannen/handprint_output /e /s google d:\GitHub\PWS-inscannen/color_correction_output/test.png 
╔═════════════════════════════════════════════════════════════════════════════╗
║           Welcome to Handprint: HANDwritten Page RecognitIoN Test           ║
╚═════════════════════════════════════════════════════════════════════════════╝
Given 1 image to work on.
Will apply results of 1 service: google
Will use credentials stored in 
C:\Users\joost\AppData\Local\CaltechLibrary\Handprint/.
Will save extended results.
Will use up to 1 process threads.
Starting on d:\GitHub\PWS-inscannen/color_correction_output/test.png
Sending to google and waiting for response ...
Got result from google.
Creating annotated image for google.
Saving all data for google.
Saving extracted text for google.
Creating results grid image: handprint_output\test.handprint-all.png
Done with color_correction_output\test.png
Done.


In [34]:
from PIL import Image, ImageDraw

output_items = os.listdir(output_dir)





def clamp(n, min, max): 
    if n < min: 
        return min
    elif n > max: 
        return max
    else: 
        return n 

for file_name in os.listdir(input_path):
    clean_file_name = file_name.split(".")
    clean_file_name.pop()
    name = ".".join(clean_file_name)
    
    output_json_file_name = name + ".handprint-google.json"
    output_image_file_name = name + ".handprint-google.png"
    if (output_json_file_name not in output_items):
        continue
    
    # output json exists
    with open(output_dir+"/"+output_json_file_name, "r") as f:
        # file_text = f.read()
        text_data = json.load(f)
    
    # print(json.dumps(text_data["text_annotations"][0], sort_keys=True, indent=4))
    horizontal_breaks = []
    for data in text_data["text_annotations"]:
        if (data["bounding_poly"]["vertices"][0]["x"] < 80):
            # if (data["description"].isnumeric()):
            horizontal_breaks.append({"height": data["bounding_poly"]["vertices"][0]["y"], "description": data["description"]})
    print(horizontal_breaks)
    
    image = Image.open(input_path+"/"+file_name)
    output_image = ImageDraw.Draw(image)
    for h_break in horizontal_breaks:
        y = h_break["height"]
        output_image.line([(0, y-15), (image.width, y-15)], fill="red", width=3,)
    
    # image.show()
    image.save(section_output+"/"+file_name)
    
    for index, h_break in enumerate(horizontal_breaks):
        if (index == 0 ):
            continue
        previous_y = clamp(horizontal_breaks[index-1]["height"] - 15, 0, image.height - 1)
        y = clamp(h_break["height"] - 15, 0, image.height - 1)
        
        # min height and fix
        if (previous_y > y or y - previous_y < 30):
            continue
        crop = (0, previous_y, image.width, y)
        
        print(crop, image)
        section_image = image.copy()
        cropped = section_image.crop(crop)
        directory = section_output+"/"+name+"_sections"
        try:
            os.makedirs(directory)
        except:
            pass
        # section_image.show()
        cropped.save(directory+"/"+h_break["description"]+".png")
        


[{'height': 118, 'description': 'Opgave\nWhetheme\nexothame Reactive\n- 2 C7 +1165 +1202 >7C02 +8H120 502\n3= de hatalysator bragt de\nGeuver ings exerque Umland\nWandlagen de reactive snelhedel Snellen 15.\nLE\n120\ngeactiveerde toestand\nCHHILSTOR DEGO\n- S:\nbi hele\npindate\nWemale\ngerator\n"Pinda\'s\nReactieproduct\nCo₂ tot 120 + Soz\nVerloop readve'}, {'height': 199, 'description': '-'}, {'height': 198, 'description': '2'}, {'height': 254, 'description': '3'}, {'height': 370, 'description': 'LE'}, {'height': 569, 'description': '-'}, {'height': 572, 'description': 'S'}, {'height': 650, 'description': 'bi'}]
(0, 103, 600, 184) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 at 0x1ADD0EDB820>
(0, 183, 600, 239) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 at 0x1ADD0EDB820>
(0, 239, 600, 355) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 at 0x1ADD0EDB820>
(0, 355, 600, 554) <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=600x845 